## SVM
                        Before CV     After CV
       Training Score:  0.85026       0.88777
       Testing Score:   0.83898       0.88106

                precision    recall  f1-score   support

     CANDIDATE       0.85      0.65      0.73       523
     CONFIRMED       0.75      0.87      0.81       594
     FALSE POS       0.98      1.00      0.99      1069

     micro avg       0.88      0.88      0.88      2186
     macro avg       0.86      0.84      0.84      2186
     weighted avg    0.88      0.88      0.88      2186

## Random Forest
                        Before CV     After CV
       Training Score:  0.99497       1.0
       Testing Score:   0.87740       0.89616

  
                precision    recall  f1-score   support

     CANDIDATE       0.84      0.73      0.78       523
     CONFIRMED       0.81      0.86      0.83       594
     FALSE POS       0.97      1.00      0.98      1069

     micro avg       0.90      0.90      0.90      2186
     macro avg       0.87      0.86      0.87      2186
     weighted avg    0.89      0.90      0.89      2186

In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("cumulative.csv")
df = df.drop(columns=["rowid", "kepid", "kepoi_name", "kepler_name", "koi_pdisposition", "koi_score", "koi_tce_delivname"])
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
# df.head()
df.describe()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
count,8744.000000,8744.000000,8744.000000,8744.000000,8744.000000,8.744000e+03,8.744000e+03,8744.000000,8744.000000,8744.000000,...,8744.000000,8744.000000,8744.000000,8744.000000,8744.000000,8744.000000,8744.000000,8744.000000,8744.000000,8744.000000
mean,0.157136,0.241194,0.201167,0.125114,56.080618,1.839523e-03,-1.839523e-03,164.563271,0.009300,-0.009300,...,-161.354758,4.310223,0.120733,-0.140411,1.704566,0.352884,-0.379933,292.075061,43.828259,14.273969
std,0.363950,0.427832,0.400895,0.330867,117.385280,7.276504e-03,7.276504e-03,66.476457,0.021662,0.021662,...,72.986448,0.431557,0.132813,0.082936,5.682429,0.906364,1.810943,4.772918,3.599786,1.343509
min,0.000000,0.000000,0.000000,0.000000,0.259820,1.100000e-08,-1.568000e-01,120.515914,0.000009,-0.569000,...,-1762.000000,0.047000,0.000000,-1.207000,0.109000,0.000000,-103.825000,279.852720,36.577381,6.966000
25%,0.000000,0.000000,0.000000,0.000000,2.667824,5.286750e-06,-2.456250e-04,132.729408,0.001200,-0.010000,...,-197.000000,4.215750,0.043000,-0.195000,0.826000,0.125750,-0.247000,288.670237,40.805911,13.474000
50%,0.000000,0.000000,0.000000,0.000000,8.970985,3.323000e-05,-3.323000e-05,136.910235,0.004020,-0.004020,...,-159.000000,4.439000,0.070000,-0.127000,0.997000,0.246000,-0.111000,292.285005,43.703989,14.534000
75%,0.000000,0.000000,0.000000,0.000000,34.190033,2.456250e-04,-5.286750e-06,169.975942,0.010000,-0.001200,...,-112.000000,4.544000,0.149000,-0.087000,1.346250,0.356000,-0.069000,295.900510,46.722135,15.318250
max,1.000000,1.000000,1.000000,1.000000,1071.232624,1.568000e-01,-1.100000e-08,1472.522306,0.569000,-0.000009,...,0.000000,5.364000,1.472000,0.000000,180.013000,33.091000,0.000000,301.720760,52.336010,19.065000


# Create a Train Test Split

Use `koi_disposition` for the y values

In [3]:
target = df["koi_disposition"]
data = df.drop("koi_disposition", axis=1)
feature_names = data.columns
data.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,-0.002160,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,-0.000581,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,-0.001130,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [5]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
8017,0,1,1,0,0.806277,4.947000e-06,-4.947000e-06,131.785670,0.006720,-0.006720,...,-184.0,4.471,0.054,-0.229,0.996,0.324,-0.108,290.81723,38.539120,13.614
1233,0,1,1,0,3.582077,4.318000e-06,-4.318000e-06,355.515064,0.000864,-0.000864,...,-235.0,4.422,0.090,-0.195,0.993,0.283,-0.131,296.07822,43.136940,15.193
2592,0,0,0,0,5.060923,2.616000e-05,-2.616000e-05,134.473160,0.004730,-0.004730,...,-112.0,4.492,0.048,-0.112,0.911,0.121,-0.060,289.91742,40.828606,13.346
4770,0,1,0,1,8.480304,3.320000e-07,-3.320000e-07,135.854534,0.000031,-0.000031,...,-169.0,3.946,0.195,-0.105,2.210,0.375,-0.563,298.80020,46.665539,7.631
6632,0,0,0,1,4.994716,4.495000e-05,-4.495000e-05,136.183300,0.009500,-0.009500,...,-194.0,3.706,0.320,-0.080,2.830,0.458,-1.068,282.58215,46.815510,13.352


# Pre-processing

Scale the data using the MinMaxScaler

In [6]:
from sklearn.preprocessing import MinMaxScaler
X_minmax = MinMaxScaler().fit(X_train)
# y_minmax = MinMaxScaler().fit(y_train)

X_train_minmax = X_minmax.transform(X_train)
X_test_minmax = X_minmax.transform(X_test)
# y_train_minmax = y_minmax.transform(y_train)
# y_test_minmax = y_minmax.transform(y_test)

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


# Train the Support Vector Machine

In [7]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train_minmax, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [8]:
print(f"Training Data Score: {model.score(X_train_minmax, y_train)}")
print(f"Testing Data Score: {model.score(X_test_minmax, y_test)}")
# print('Test Acc: %.3f' % model.score(X_test, y_test))

Training Data Score: 0.8502592253735896
Testing Data Score: 0.838975297346752


# Hyperparameter Tuning

Use `GridSearchCV` to tune the `C` and `gamma` parameters

In [9]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [10]:
# Train the model with GridSearch
grid.fit(X_train_minmax, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] C=1, gamma=0.0001 ...............................................


/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ....... C=1, gamma=0.0001, score=0.846892138939671, total=   0.3s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV] ...... C=1, gamma=0.0001, score=0.8384439359267735, total=   0.3s
[CV] C=1, gamma=0.0001 ...............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.1s remaining:    0.0s


[CV] ...... C=1, gamma=0.0001, score=0.8480549199084668, total=   0.3s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ....... C=1, gamma=0.0005, score=0.846892138939671, total=   0.3s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ...... C=1, gamma=0.0005, score=0.8384439359267735, total=   0.3s
[CV] C=1, gamma=0.0005 ...............................................
[CV] ...... C=1, gamma=0.0005, score=0.8480549199084668, total=   0.3s
[CV] C=1, gamma=0.001 ................................................
[CV] ........ C=1, gamma=0.001, score=0.846892138939671, total=   0.3s
[CV] C=1, gamma=0.001 ................................................
[CV] ....... C=1, gamma=0.001, score=0.8384439359267735, total=   0.3s
[CV] C=1, gamma=0.001 ................................................
[CV] ....... C=1, gamma=0.001, score=0.8480549199084668, total=   0.3s
[CV] C=1, gamma=0.005 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:   25.6s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [1, 5, 10, 50], 'gamma': [0.0001, 0.0005, 0.001, 0.005]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [11]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 50, 'gamma': 0.0001}
0.8824336688014639


In [12]:
grid.score(X_train_minmax, y_train)

0.8877706617871303

In [13]:
grid.score(X_test_minmax, y_test)

0.8810612991765783

In [14]:
predictions = grid.predict(X_test_minmax)

In [15]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

                precision    recall  f1-score   support

     CANDIDATE       0.85      0.65      0.73       523
     CONFIRMED       0.75      0.87      0.81       594
FALSE POSITIVE       0.98      1.00      0.99      1069

     micro avg       0.88      0.88      0.88      2186
     macro avg       0.86      0.84      0.84      2186
  weighted avg       0.88      0.88      0.88      2186



# Random Forest

In [16]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf = rf.fit(X_train_minmax, y_train)
print(rf.score(X_train_minmax, y_train))
print(rf.score(X_test_minmax, y_test))

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.994967978042086
0.8774016468435498


In [17]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.11741799834178808, 'koi_fpflag_co'),
 (0.08111972015401982, 'koi_fpflag_nt'),
 (0.06388975376868553, 'koi_prad'),
 (0.05388501700451346, 'koi_fpflag_ss'),
 (0.05211873399229984, 'koi_model_snr'),
 (0.0500684793913244, 'koi_duration_err1'),
 (0.04689897342776127, 'koi_fpflag_ec'),
 (0.045573516329538946, 'koi_duration_err2'),
 (0.04001359444738137, 'koi_depth'),
 (0.026934364707184837, 'koi_impact'),
 (0.026642909511897732, 'koi_steff_err2'),
 (0.02612458198293714, 'koi_duration'),
 (0.025280195223127466, 'koi_prad_err1'),
 (0.022738130814527892, 'koi_period'),
 (0.022651842191170495, 'koi_depth_err2'),
 (0.019277151280475625, 'koi_time0bk_err2'),
 (0.01897108559673557, 'koi_insol_err1'),
 (0.018640241262357436, 'koi_teq'),
 (0.014977317872848098, 'koi_time0bk_err1'),
 (0.014896666665169377, 'koi_period_err1'),
 (0.014173296843137706, 'koi_steff_err1'),
 (0.01412256909958618, 'koi_prad_err2'),
 (0.013419308496726876, 'koi_time0bk'),
 (0.013303459383466979, 'koi_insol'),
 (0.01317636

In [46]:
param_grid2 = {'n_estimators': [250, 300, 350],
              'max_depth': [125, 150, 175]}
grid2 = GridSearchCV(rf, param_grid2, verbose=3)

In [47]:
grid2.fit(X_train_minmax, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] max_depth=125, n_estimators=250 .................................
[CV]  max_depth=125, n_estimators=250, score=0.8912248628884827, total=   3.0s
[CV] max_depth=125, n_estimators=250 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.1s remaining:    0.0s


[CV]  max_depth=125, n_estimators=250, score=0.8851258581235698, total=   2.9s
[CV] max_depth=125, n_estimators=250 .................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.1s remaining:    0.0s


[CV]  max_depth=125, n_estimators=250, score=0.8956521739130435, total=   2.9s
[CV] max_depth=125, n_estimators=300 .................................
[CV]  max_depth=125, n_estimators=300, score=0.8930530164533821, total=   3.5s
[CV] max_depth=125, n_estimators=300 .................................
[CV]  max_depth=125, n_estimators=300, score=0.8860411899313501, total=   3.4s
[CV] max_depth=125, n_estimators=300 .................................
[CV]  max_depth=125, n_estimators=300, score=0.8965675057208238, total=   3.5s
[CV] max_depth=125, n_estimators=350 .................................
[CV]  max_depth=125, n_estimators=350, score=0.8939670932358318, total=   4.1s
[CV] max_depth=125, n_estimators=350 .................................
[CV]  max_depth=125, n_estimators=350, score=0.8846681922196796, total=   4.0s
[CV] max_depth=125, n_estimators=350 .................................
[CV]  max_depth=125, n_estimators=350, score=0.9006864988558353, total=   4.0s
[CV] max_depth=150, n

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  1.6min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [250, 300, 350], 'max_depth': [125, 150, 175]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [48]:
print(grid2.best_params_)
print(grid2.best_score_)

{'max_depth': 125, 'n_estimators': 350}
0.8931076547727966


In [49]:
grid2.score(X_train_minmax, y_train)

1.0

In [50]:
grid2.score(X_test_minmax, y_test)

0.8961573650503202

In [51]:
predictions2 = grid2.predict(X_test_minmax)

In [52]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions2))

                precision    recall  f1-score   support

     CANDIDATE       0.84      0.73      0.78       523
     CONFIRMED       0.81      0.86      0.83       594
FALSE POSITIVE       0.97      1.00      0.98      1069

     micro avg       0.90      0.90      0.90      2186
     macro avg       0.87      0.86      0.87      2186
  weighted avg       0.89      0.90      0.89      2186

